In [1]:
import os
import time
import json

import numpy as np
import skimage
import tifffile 

import keras
from keras.models import load_model
from keras.utils import CustomObjectScope
from keras.initializers import glorot_uniform

from dice import dice_coef, dice_loss
from predict import predict_mask
from visualize import display_slice_from_batch

/Users/dnadeem/Desktop/dl/pyLattice_deepLearning/src/predict.py:35: SyntaxWarning: "is" with a literal. Did you mean "=="?
  assert offset.size is 3, "Offset array needs to have a size of 3."


In [2]:
image_path = "/Users/dnadeem/Desktop/dl/pyLattice_deepLearning/src/quickstart-data/train_data_input.tif"
truth_path = "/Users/dnadeem/Desktop/dl/pyLattice_deepLearning/src/quickstart-data/mask_all_frames_modified.tif"
model_path = "jan30_largeframe_32_nonzero_standardized_local.keras"

# Use the same patch_size that was used for training
patch_size = 32

In [3]:
with CustomObjectScope({'GlorotUniform': glorot_uniform()}):
        model = load_model(model_path, custom_objects={'dice': dice_loss(), 'dice_coef': dice_coef})

2024-06-08 21:43:24.425613: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M3 Pro
2024-06-08 21:43:24.425665: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 36.00 GB
2024-06-08 21:43:24.425678: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 13.50 GB
2024-06-08 21:43:24.425718: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-06-08 21:43:24.425741: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [4]:
# Uses the overloaded predict_mask method which can detect the correct size
start = time.time()
mask = predict_mask(model, image_path, patch_size, offset=np.asarray([0,0,0]))
end = time.time()
print("Time to complete: " + str(end - start))

Image cropped to: 256, 256, 32


/Users/dnadeem/Desktop/dl/pyLattice_deepLearning/src/predict.py:35: SyntaxWarning: "is" with a literal. Did you mean "=="?
  assert offset.size is 3, "Offset array needs to have a size of 3."
/Users/dnadeem/Desktop/dl/pyLattice_deepLearning/src/predict.py:35: SyntaxWarning: "is" with a literal. Did you mean "=="?
  assert offset.size is 3, "Offset array needs to have a size of 3."


IndexError: too many indices for array: array is 3-dimensional, but 4 were indexed

In [ ]:
# View your prediction file

import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [30, 30]
plt.set_cmap('gray')
z=5

display_slice_from_batch(tifffile.imread(image_path)[:mask.shape[1], :mask.shape[2], :mask.shape[3]].reshape(mask.shape), z=z)
display_slice_from_batch(tifffile.imread(truth_path)[:mask.shape[1], :mask.shape[2], :mask.shape[3]].reshape(mask.shape), z=z)
display_slice_from_batch(mask, z=z)

In [ ]:
# Run the following if you intend to DOWNLOAD the predicted tif file (optional)
prediction_file = (mask*255.0).astype('uint16')
metadata = dict(microscope='joh', shape=prediction_file.shape, dtype=prediction_file.dtype.str)
metadata = json.dumps(metadata)

# Change name of file
skimage.external.tifffile.imsave("Production2_PSNR100prediction.tif", prediction_file, description=metadata)    